# SSVEP classification comparison

This notebook takes the SSVEP epochs and the no stim periods and used a classifier.

## Import libraries

In [9]:
# Standard libraries
import numpy as np

# Custom libraries
from Functions import import_data
from Functions import data_tools
from Functions import processing


## Import and epoch data

In [10]:
# Settings
files = [  
    "sub-P004_ses-S001_task-T1_run-001_eeg"
    # "sub-P002_ses-S001_task-T1_run-001_eeg"   
]
ch_names = ["O1", "Oz" ,"O2"] # List of channel names to import
fc = [1, 35]            # Cut-off frequencies for BW filter

# Isolate subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables
eeg_epochs = [None] * len(files)
settings = [None] * len(files)

for f, file in enumerate(files):
    # Import data and markers
    [eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(fr"Data\\{file}.xdf", picks=ch_names)  
    [marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\{file}.xdf")

    # Filter data
    eeg_filt = processing.butter_filt(eeg_data, fc, "bandpass", eeg_fs)

    # Create epochs from Unity markers
    (eeg_epochs, epoch_labels) = data_tools.epochs_from_unity_markers(
        eeg_time = eeg_ts,
        eeg_data = eeg_filt,
        marker_time = marker_ts,
        marker_data = markers
        )

    # Fix mispelled labels & create numeric code for them
    fixed_labels = data_tools.fix_labels(epoch_labels) 

    #drop resting state labels and epochs
    labels_to_drop = ["Resting state, eyes open", "Resting state, eyes closed"]
    drop_rs_epochs, drop_rs_labels = data_tools.drop_epochs_by_label(eeg_epochs, fixed_labels, labels_to_drop)

    [label_dict, label_array] = data_tools.labels_to_dict_and_array(drop_rs_labels)
    

<string>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\{'
<string>:20: SyntaxWarning: invalid escape sequence '\{'
<>:20: SyntaxWarning: invalid escape sequence '\{'
C:\Users\danie\AppData\Local\Temp\ipykernel_27012\3025111396.py:20: SyntaxWarning: invalid escape sequence '\{'
  [marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\{file}.xdf")


In [11]:
# Trim epochs so that they can be in numpy array instead of list
trimmed_epochs = data_tools.normalize_epochs_length(drop_rs_epochs, "zeropad")
#time base epoch by 4 seconds 
#"off" = 2 4 second
#"on" = 3 4 second 

#either 0 pad the off into 12 seconds or trim epochs


## Riemmanian geometry classification imbalanced classes

In [12]:
import scipy
import sklearn
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from imblearn.over_sampling import ADASYN

# Create classifier
pipelines_fb = make_pipeline(
    Covariances(estimator="lwf"),
    TangentSpace(),
    LogisticRegression(solver="lbfgs", multi_class="auto"),
    )

# Oversample stimulus classes
sm = ADASYN(
    random_state = 42, 
    n_neighbors = 2     #2 id there are 3 samples of "on"
)
[nepochs, nchans, nsamples] = np.shape(trimmed_epochs)
reshaped_epochs = np.reshape(trimmed_epochs, [nepochs, nchans*nsamples])
[x_oversampled, y_oversampled] = sm.fit_resample(reshaped_epochs, label_array)

# - Reshape oversampled data to be [epoch, channel, samples]
x_oversampled_unfold = np.reshape(
    x_oversampled,
    [x_oversampled.shape[0], nchans, -1]
    )

# Split data
[X_train, X_test, y_train, y_test] = train_test_split(x_oversampled_unfold, y_oversampled, random_state= 4)

# Test classifier
pipelines_fb.fit(X_train, y_train).score(X_test, y_test)

0.7355473554735548

In [13]:
y_pred = pipelines_fb.predict(X_test)
cm_all = cm(y_test, y_pred)

## Single stimulus type classification

This section takes a string to select the type of stimulus you want and it does the same Riemmanian classification as above using the `stimulus_of_interest` and the `Stimulus Off` events that precede `stimulus_of_interest`.

In [17]:
# Settings
stimulus_of_interest = " Voronoi"   # Note that there is a space before the name

# Create mask and labels for stimulus of interest
(single_labels, bool_mask) = data_tools.single_stimulus_mask(drop_rs_labels, " SquareGrate")
single_stimulus_epochs = trimmed_epochs[bool_mask, : , :]

# Reshape data to solve imbalance in classes
[nepochs, nchans, nsamples] = np.shape(single_stimulus_epochs)
reshaped_epochs = np.reshape(single_stimulus_epochs, [nepochs, nchans*nsamples])
[x_oversampled, y_oversampled] = sm.fit_resample(reshaped_epochs, single_labels)

# Reshape oversampled data to be [epoch, channel, samples]
x_oversampled_unfold = np.reshape(
    x_oversampled,
    [x_oversampled.shape[0], nchans, -1]
    )

# Split data
[X_train_single, X_test_single, y_train_single, y_test_single] = train_test_split(
    x_oversampled_unfold,
    y_oversampled,
    random_state= 4)

# Test classifier
pipelines_fb.fit(X_train_single, y_train_single).score(X_test_single, y_test_single)

0.9

In [16]:
y_pred_single = pipelines_fb.predict(X_test_single)
cm_single = cm(y_test_single, y_pred_single)